In [1]:
import os

ספריות נדרשות עבור הרצת הפרוייקט

In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
import time
# store background images in a list
image_path = os.getcwd()+'/Images/'
images = os.listdir(image_path)

יצירת מסלול עבור התמונה אותה נרצה לשים בתור רקע 
אינדקס 0 אומר התמונה במקום ה0 מתוך התיקיה ץ אם נרצה תמונה אחרת נחליף את האינדקס

In [3]:
image_index= 0
bg_image = cv2.imread(image_path+'/'+images[image_index])
cv2.imshow("Frame", bg_image)

In [4]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

פונקציות אלו יוצרות אובייקט מתוך cv2

כל זאת על מנת שנוכל להשתמש בפונקציונליות של הספריה המבוקשת

rgb - תמונה מורכבת מ3 שכבות 
אדום r
ירוק g
כחול b 
התמונות הן בעצם מערך של מספרים מ0 עד 255 עבורן כל ערך מגדיר (כמקשה אחת מורכבת) את התמונה עצמה 

כאשר 0 - נטול צבע באותו ערך  ו 255 זה הערך הגבוהה ביותר.

כאן אנחנו מבקשים לפתוח את המצלמהץ

כל עוד המצלמה פתוחה (תנאי הwhile( :

אנחנו נשתמש באורך וגובה _ונהפוך אותן לפי האקס שלו ץ

לאחר מכן נבצע מסקינג על מנת שנחליף את התמונה בהתאם 

נבקש להמתין בעזרת הוויט 

לאחר מכן אנחנו נרצה את אותם המימדים של התמונה המקורית עבור הרקע אחרת לא נקבל תוצאה רצוינ (כל זאת עבור התמונה המבוקשת - כזכור התמונה הראשונה באינדקס 0)

נעשה שינוי מימד כמו שנדרשים

ונבקש להציג את התמונה 

התוצאה תתן לנו את ההרגשה של החלפת רקע כאשר הביצוע היה לשים תמונה שתרוץ איתנו כל עוד המלצמה דלוקה ופתוחה 

בכדי להחליף תמונה אנו נדרשים רק ללחוץ על d או על האות q על מנת לעצור את התהליך ולצאת 

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
  # flip the frame to horizontal direction
    frame = cv2.flip(frame, 1)
    height , width, channel = frame.shape
    RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  # get the result
    results = selfie_segmentation.process(RGB)
  # extract segmented mask
    mask = results.segmentation_mask
  # show outputs
    cv2.imshow("mask", mask)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    condition = np.stack(
    (results.segmentation_mask,) * 3, axis=-1) > 0.5
  # resize the background image to the same size of the original frame
    bg_image = cv2.resize(bg_image, (width, height))
  # combine frame and background image using the condition
    output_image = np.where(condition, frame, bg_image)
    cv2.imshow("Output", output_image)
    cv2.imshow("Frame", frame)
    time.sleep(2)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
  # if 'd' key is pressed then change the background image
    elif key == ord('d'):
        if image_index != len(images)-1:
              image_index += 1
        else:
              image_index = 0
        bg_image = cv2.imread(image_path+'/'+images[image_index])
